# Download counts for nteract

In [ ]:
import IPython.display
import pandas as pd

In [ ]:
import requests

# Note: 
data = requests.get('https://api.github.com/repos/nteract/nteract/releases').json()

In [ ]:
print("{}:\n\t{}\n\t{}".format(
  data[0]['tag_name'],
  data[0]['assets'][0]['browser_download_url'],
  data[0]['assets'][0]['download_count']
))


The releases API only has context of the filename, so we'll convert:

```
https://github.com/nteract/nteract/releases/download/v0.0.13/nteract-darwin-x64.zip
```

to

```
darwin-x64
```

Which means we're reliant on our release naming to keep this a nice consistent structure

In [ ]:
def strip_off_release(browser_download_url):
    filename = browser_download_url.split('/')[-1]
    basename = filename.split('.')[0]
    system = basename.split('-')[1:]
    return "-".join(system)

In [ ]:
releases = [
  {
    'version': x['tag_name'], 
    'counts': { strip_off_release(y['browser_download_url']): y['download_count'] for y in x['assets'] }
  } 
    for x in data
]
releases

In [ ]:
versions = []
frames = []

for release in releases:
    versions.append(release['version'])
    frames.append(pd.DataFrame.from_dict(release['counts'], orient='index'))

df = pd.concat(frames, keys=versions).reset_index()
df.columns = ['version', 'os', 'count']
df['os'] = df.os.replace('os-x', 'darwin-x64')
df

It would be really interesting to know how these counts change over time.

In [ ]:
from distutils.version import LooseVersion

versions = set(df.version.values.tolist())
versions = sorted(versions, key=LooseVersion)

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline
%config InlineBackend.figure_format = 'retina'

with sns.color_palette("colorblind", len(versions)):
    fig = plt.figure(figsize=(10, 6))
    ax = fig.add_subplot(1, 1, 1)
    ax = sns.barplot(x='version', y="count", hue="os", data=df, order=versions)
    ax.set_xticklabels(versions, rotation=30)
    ax.set(xlabel='Version', ylabel='Count')
    plt.legend(bbox_to_anchor=(1, 1), loc=2, borderaxespad=0)
    plt.show()